In [107]:
import numpy as np
import pandas as pd
url = '/home/kenwu/Documents/ML_Zoomcamp/data.csv'

df = pd.read_csv(url)

In [108]:
df = df[['neighbourhood_group',
'room_type',
'latitude',
'longitude',
'price',
'minimum_nights',
'number_of_reviews',
'reviews_per_month',
'calculated_host_listings_count',
'availability_365']]
df['reviews_per_month'].fillna(0)

0        0.21
1        0.38
2        0.00
3        4.64
4        0.10
         ... 
48890    0.00
48891    0.00
48892    0.00
48893    0.00
48894    0.00
Name: reviews_per_month, Length: 48895, dtype: float64

In [109]:
df['neighbourhood_group'].describe()

count         48895
unique            5
top       Manhattan
freq          21661
Name: neighbourhood_group, dtype: object

In [110]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

df_full_train['price_binary']= df_full_train['price'].apply(lambda x: 1 if (x >= df['price'].mean()) else 0)
df_test['price_binary']= df_test['price'].apply(lambda x: 1 if (x >= df['price'].mean()) else 0)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

y_train = df_train.price_binary.values
y_val = df_val.price_binary.values
y_test = df_test.price_binary.values

del df_train['price']
del df_train['price_binary']
del df_val['price']
del df_val['price_binary']
del df_test['price']
del df_test['price_binary']


In [111]:
numerical = ['latitude','longitude','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count', 'availability_365']
categorical = ['neighbourhood_group','room_type']
df_numerical = df[numerical]
df_numerical.corr()

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.084788,0.024869,-0.015389,-0.010142,0.019517,-0.010983
longitude,0.084788,1.000000,-0.062747,0.059094,0.145948,-0.114713,0.082731
minimum_nights,0.024869,-0.062747,1.000000,-0.080116,-0.121702,0.127960,0.144303
number_of_reviews,-0.015389,0.059094,-0.080116,1.000000,0.549868,-0.072376,0.172028
reviews_per_month,-0.010142,0.145948,-0.121702,0.549868,1.000000,-0.009421,0.185791
calculated_host_listings_count,0.019517,-0.114713,0.127960,-0.072376,-0.009421,1.000000,0.225701
availability_365,-0.010983,0.082731,0.144303,0.172028,0.185791,0.225701,1.000000


In [112]:
from sklearn.metrics import mutual_info_score
print(mutual_info_score(df_full_train.price_binary, df_full_train.neighbourhood_group).round(3))
print(mutual_info_score(df_full_train.price_binary, df_full_train.room_type).round(3))

0.046
0.144


In [113]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [121]:
X_train[0]

array([ 3.630000e+02,  1.000000e+00,  4.076434e+01, -7.392132e+01,
        4.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
        1.000000e+00,  0.000000e+00,  3.000000e+01,  3.200000e-01,
        1.000000e+00,  0.000000e+00,  0.000000e+00])

In [114]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').